# Generating An Instruction Dataset via Llama 3 and Ollama

- This notebook uses an 8-billion-parameter Llama 3 model through ollama to generate a synthetic dataset using the "hack" proposed in the "Magpie: Alignment Data Synthesis from Scratch by Prompting Aligned LLMs with Nothing" paper ([https://arxiv.org/abs/2406.08464](https://arxiv.org/abs/2406.08464))

- The generated dataset will be an instruction dataset with "instruction" and "output" field similar to what can be found in Alpaca:


```python
{
    "instruction": "What is the atomic number of helium?",
    "output": "The atomic number of helium is 2.",
},
```

- The code doesn't require a GPU and runs on a laptop (it was tested on a M3 MacBook Air)

*Note that the instruction datasets created here are for educational purposes. However, it is the users' duty to ensure that their use adheres to the terms of the relevant licensing agreements with Meta AI's Llama 3.*

In [ ]:
from importlib.metadata import version

pkgs = [
    "tqdm",    # Progress bar
]

for p in pkgs:
    print(f"{p} version: {version(p)}")

## Installing Ollama and Downloading Llama 3

- Ollama is an application to run LLMs efficiently
- It is a wrapper around [llama.cpp](https://github.com/ggerganov/llama.cpp), which implements LLMs in pure C/C++ to maximize efficiency
- Note that it is a tool for using LLMs to generate text (inference), not training or finetuning LLMs
- Prior to running the code below, install ollama by visiting [https://ollama.com](https://ollama.com) and following the instructions (for instance, clicking on the "Download" button and downloading the ollama application for your operating system)

- For macOS and Windows users, click on the ollama application you downloaded; if it prompts you to install the command line usage, say "yes"
- Linux users can use the installation command provided on the ollama website

- In general, before we can use ollama from the command line, we have to either start the ollama application or run `ollama serve` in a separate terminal

<img src="https://raw.githubusercontent.com/Sangwan70/Building-an-LLM-From-Scratch/refs/heads/main/part_6/images/ollama-serve.webp?1">


- With the ollama application or `ollama serve` running, in a different terminal, on the command line, execute the following command to try out the 3-billion-parameter Llama 3.2 model (the model, which takes up 4.7 GB of storage space, will be automatically downloaded the first time you execute this command)

```bash
ollama run llama3.2
```


The output looks like as follows:

```
$ ollama run llama3.2
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
```

- Note that `llama3.2` refers to the instruction finetuned 3-billion-parameter Llama 3.2 model

- After the download has been completed, you will see a command line prompt that allows you to chat with the model

- Try a prompt like "What do llamas eat?", which should return an output similar to the following:

```
% ollama run llama3.2
>>> What are the courses offered at Oracle University? 
Oracle University offers a wide range of courses and certifications in various areas, including:

1. Oracle Database Administration:
 * Oracle Database: Essentials (10g/11g)
 * Oracle Database: Advanced (10g/11g)
 * Oracle Database: Performance Tuning
2. Oracle E-Business Suite:
 * Oracle E-Business Suite: Essentials (12c/13c)
 * Oracle E-Business Suite: Advanced (12c/13c)
 * Oracle E-Business Suite: Implementation and Development

```

- You can end this session using the input `/bye`

## Using Ollama's REST API

- Now, an alternative way to interact with the model is via its REST API in Python via the following function
- Before you run the next cells in this notebook, make sure that ollama is still running, as described above, via
  - `ollama serve` in a terminal
  - the ollama application
- Next, run the following code cell to query the model

- First, let's try the API with a simple example to make sure it works as intended:

In [1]:
import urllib.request
import json

def query_model(prompt, model="llama3.2", url="http://localhost:11434/api/chat", role="user"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "seed": 123,        # for deterministic responses
        "temperature": 1.,   # for deterministic responses
        "top_p": 1,         
        "messages": [
            {"role": role, "content": prompt}
        ]
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [2]:
result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based materials. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even aquatic plants.
2. Hay: Hay is a staple in a llama's diet, especially during the winter months when other food sources may be scarce.
3. Grains: Llamas can also eat grains like oats, barley, and corn, but these should not make up more than 10% of their diet.
4. Browse: Browse refers to shrubs, leaves, and other woody vegetation that are high in fiber and nutrients. Llamas enjoy browsing on plants like alfalfa, clover, and sweetgum trees.
5. Fruits and vegetables: In moderation, llamas can eat fruits and vegetables, such as apples, carrots, and squash.

It's essential to note that llamas have a specific nutritional requirement, so it's best to provide them with a balanced diet that includes a mix of these food sources.

Here are some guidelines for feeding l

## Extract Instructions

- Now, let's use the "hack" proposed in the paper: we provide the empty prompt template `"<|begin_of_text|><|start_header_id|>user<|end_header_id|>"` prompt, which will cause the instruction-finetuned Llama 3 model to generate an instruction

In [3]:
def extract_instruction(text):
    for content in text.split("\n"):
        if content:
            return content.strip()

In [4]:
query = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query, role="assistant")
instruction = extract_instruction(result)
print(instruction)

I've been experiencing some issues with my computer's Wi-Fi connectivity. I'm having trouble connecting to a particular network. Here are the details of the issue:


- As we can see above, surprisingly, the model indeed generated an instruction

## Generate Responses

- Now, the next step is to create the corresponding response, which can be done by simply passing the instruction as input

In [5]:
response = query_model(instruction, role="user")
print(response)

I'd be happy to help you troubleshoot your Wi-Fi connectivity issues. Please go ahead and share the details of the issue, such as:

* The name or password of the network you're trying to connect to
* What device (computer, phone, etc.) you're using to try and connect
* Any error messages you've received when attempting to connect
* What kind of Wi-Fi router you have and its make/model
* If there are any other devices connected to the same network that are working fine

The more information you can provide, the better I'll be able to assist you in resolving your issue.


## Generate Dataset

- We can scale up this approach to an arbitrary number of data samples (you may want to apply some optional filtering length or quality (e.g., using another LLM to rate the generated data)
- Below, we generate 5 synthetic instruction-response pairs, which takes about 3 minutes on an M3 MacBook Air
- (To generate a dataset suitable for instruction finetuning, we want to increase this to at least 1k to 50k and perhaps run it on a GPU to generate the examples in a more timely fashion)

In [6]:
from tqdm import tqdm

dataset_size = 5
dataset = []

for i in tqdm(range(dataset_size)):

    result = query_model(query, role="assistant")
    instruction = extract_instruction(result)
    response = query_model(instruction, role="user")
    entry = {
        "instruction": instruction,
        "output": response
    }
    dataset.append(entry)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.28s/it]


In [7]:
with open("instruction-data-llama3_2-3b.json", "w") as file:
    json.dump(dataset, file, indent=4)

In [8]:
!cat instruction-data-llama3_2-3b.json

[
    {
        "instruction": "Hello! I need help with a math problem. Can you explain how to solve it?",
        "output": "I'd be happy to help you with your math problem. What's the specific problem you're working on? Please share the numbers and the operation (addition, subtraction, multiplication, or division) involved, and I'll guide you through the solution step by step."
    },
    {
        "instruction": "I'm looking for information on the use of social media in the workplace. Here are some specific questions I'd like answers to:",
        "output": "I'd be happy to help you with your questions about using social media in the workplace. Please go ahead and ask your questions, and I'll do my best to provide you with accurate and helpful information.\n\n(And just to confirm, I'll assume that you're looking for general guidance and not seeking specific advice tailored to a particular organization or individual. If you have specific context or requirements, feel free to let me k